In [2]:
from bs4 import BeautifulSoup
import feedparser, urllib, json, os, re, requests
from collections import defaultdict

In [2]:
#soup.find_all('h3', {'class': 'StyledHeading-sc-1rdh4aw-0 haGwpv'})

def grab_ratings(brand_page_soup):

    #regrex patterns
    rate_pat = re.compile('\d')
    cost_pat = re.compile('[$]')

    brand_ratings = defaultdict(dict)
    brand_metrics = brand_page_soup.find_all('span', {'class': 'StyledText-sc-1sadyjn-0 ccIhDL'})

    #get second item, first item is the aggregated "rating", then count the number of '$' to rank it
    brand_cost = brand_page_soup.find_all('span', {'class': 'StyledText-sc-1sadyjn-0 kkXGYR'})[1].text
    brand_cost_rank = len(re.findall(cost_pat, brand_cost))


    for i, metric in enumerate(['plant', 'people', 'animals']):
        rating = re.findall(rate_pat, brand_metrics[i].text)
        if not rating:
            brand_ratings[metric] = None
        else:
            rating_dec = int(rating[0])/int(rating[1])
            brand_ratings[metric] = rating_dec
    
    return(brand_cost_rank, brand_ratings)

In [28]:
#MAIN "TOPS"

tops_site = 'https://directory.goodonyou.eco/categories/tops'
hdr = {'User-agent': 'Mozilla/5.0'}
req = urllib.request.Request(tops_site,headers=hdr)

tops_page = urllib.request.urlopen(req)
tops_soup = BeautifulSoup(tops_page, 'html.parser')
entire_page_meta = json.loads(tops_soup.script.string)

all_brands = entire_page_meta['props']['pageProps']['category']['brands']

tops_brand_page = {'https://directory.goodonyou.eco/brand/'+i['id'] for i in all_brands}

In [8]:
#STEPPING INTO EACH BRAND PAGE
t_shirt_brand_dict = {}
for brand_site in tops_brand_page:
    
    req2 = urllib.request.Request(brand_site,headers=hdr)

    try:
        page2 = urllib.request.urlopen(req2)
    except:
        t_shirt_brand_dict[brand_site] = 'NOT FOUND'
        continue
  #  page2 = urllib.request.urlopen(req2)
    soup2 = BeautifulSoup(page2, 'html.parser')
    brand_cost, brand_rate = grab_ratings(soup2)
    
    
    brand = json.loads(soup2.script.string)
    brand_meta = brand['props']['pageProps']['brand']
    t_shirt_check=False
    for cat_i in brand_meta['categories']:
        if cat_i['name']=='T-Shirts':
            t_shirt_check = True

    if t_shirt_check:
        pass
    else:
        continue
    t_shirt_brand_dict[brand_meta['id']] = {'link':brand_site, 'cost':brand_cost, 'rating':brand_rate, 'extra_meta':brand_meta}


In [12]:
# note that output.json must already exist at this point
with open('data/brand_data.json', 'w+') as f:
    # this would place the entire output on one line
    # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(t_shirt_brand_dict, f)

In [5]:
with open('data/brand_data.json', 'r') as f:
    loaded = json.load(f)

In [ ]:
#brand name: {link:(http://...), cost:(1,2,3,4), ratings{planet:(1-5), 'people':(1-5), 'animals':(1-5)}, additional metadata: {...}}

# Trying to go deeper into other sites in a general way

In [46]:
tst_site = loaded['sandro']['extra_meta']['website']
hdr = {'User-agent': 'Mozilla/5.0'}
req = urllib.request.Request(tst_site,headers=hdr)

tops_page = urllib.request.urlopen(req)
tops_soup = BeautifulSoup(tops_page, 'html.parser')

In [47]:
for link_i in tops_soup.find_all('a'):
    print(link_i.text)
    
    
    #, {'text':'Women'})#[1].text

Women
Clothing
Shoes
Bags & Accessories











Men
Clothing
Shoes
Accessories











FAQ
Contact Us
Our Stores
Women
Men
Australia
Austria
Belgium
China
France
Germany
Greece
Hong Kong SAR / Macau SAR
Ireland
Italy
Luxembourg
Netherlands
Portugal
Russia
Singapore
Spain
Switzerland
Taiwan Region
United Kingdom
